In [3]:
import torch
import torchaudio
import os
import numpy as np
import pandas as pd
import json

In [75]:
sam_file = '../audioData/sythenticSoundscenes/test/restaurant8_downshift_0.wav'

def get_label_file(audio_filename):
    dir_name, fname = os.path.split(audio_filename)
    lab_fname = (fname.split('.')[0] + '.jams')

    return os.path.join(dir_name, lab_fname)

def get_labels(audio_filename):
    events_dict = {}
    label_fname = get_label_file(audio_filename)

    with open(label_fname, 'r') as f:
        label_data_json = json.load(f)

    for events in label_data_json['annotations'][0]['data']:
        start_time = events['value']['event_time']
        duration = events['value']['event_duration']
        events_dict[events['value']['label']] = [start_time, duration]

    return events_dict

get_labels(sam_file)

{'restaurant8': [0, 30.0],
 'laughter': [10.17445394298548, 2.150998],
 'footsteps': [10.207335203687283, 1.194671],
 'doorclose': [17.8854831473633, 1.193923],
 'speech': [18.856570541367145, 2.75],
 'chairsmoving': [22.59868571547447, 2.599751]}

In [68]:
sam_file = '../audioData/sythenticSoundscenes/test/bus1_0.jams'

with open(sam_file, 'r') as f:
    data = json.load(f)


#data = json.dumps(data)
for i in data['annotations'][0]['data']:
    print(i['value']['role'])

background
foreground
foreground
foreground
foreground
foreground
foreground
foreground
foreground
foreground
foreground
foreground
foreground
foreground
foreground
foreground
foreground


In [45]:
data

{'annotations': [{'annotation_metadata': {'curator': {'name': '', 'email': ''},
    'annotator': {},
    'version': '',
    'corpus': '',
    'annotation_tools': '',
    'annotation_rules': '',
    'validation': '',
    'data_source': ''},
   'namespace': 'scaper',
   'data': [{'time': 0.0,
     'duration': 30.0,
     'value': {'label': 'bus1',
      'source_file': 'inputSounds\\background\\train\\bus1\\bus01.wav',
      'source_time': 0,
      'event_time': 0,
      'event_duration': 30.0,
      'snr': 0,
      'role': 'background',
      'pitch_shift': None,
      'time_stretch': None},
     'confidence': 1.0},
    {'time': 8.40278405578049,
     'duration': 1.2493071687393194,
     'value': {'label': 'speech',
      'source_file': 'inputSounds\\foreground\\testing\\speech\\speech_norm_001.wav',
      'source_time': 0.0,
      'event_time': 8.40278405578049,
      'event_duration': 1.3222545819640197,
      'snr': 6.233151927278712,
      'role': 'foreground',
      'pitch_shift': 0.